<img src="res/itm_logo.jpg" width="300px">

## Inteligencia Artificial - IAI84
### Instituto Tecnológico Metropolitano
#### Pedro Atencio Ortiz - 2019

En este notebook aplicamos una red Convolutiva neuronal en Keras a un dataset de clasificación.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

In [ ]:
mnist = keras.datasets.fashion_mnist

In [ ]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
print(training_images.shape)
print(training_labels.shape)
print("Categorias del dataset: ",np.unique(training_labels))

In [ ]:
#Despleguemos ejemplos del dataset por categoria

fig = plt.figure(figsize=(10,5))
fig.suptitle("Categorias del dataset Fashion MNIST")
plt.subplots_adjust(wspace=0.5)

for j in range(10): #10 categorias
    for i in range(10000): #60000 ejemplos en el dataset de prueba
        if(test_labels[i] == j):
            plt.subplot(2,5,j+1)
            plt.title("Categoria: "+str(j))
            plt.imshow(test_images[i], cmap='gray')
            break
plt.show()

<hr>

## Convolution

Una convolucion es una operacion que consiste en "aplicar" un filtro $k$ sobre una imagen $I$, donde el filtro $k$ es una matriz de menor tamanio que la imagen $I$. La convolucion se define como:

<br>

<center>
    <font size="5">
        
    $G(x,y) = k * I(x,y) = \sum^{a}_{s=-a}\sum^{b}_{t=-b}{k(s,t)*I(x-s, y-t)}$
   
   </font>
</center>

In [ ]:
from scipy import signal

k1 = np.array([[-1,-1,-1],[0,0,0],[1,1,1]])
k2 = k1.T

I = training_images[np.random.randint(0,60000)]
G1 = signal.correlate2d(I,k1)
G2 = signal.correlate2d(I,k2)


plt.figure(figsize=(10,7))

plt.subplot(231)
plt.imshow(I, cmap='gray')
plt.subplot(232)
plt.imshow(k1, cmap='gray')
plt.subplot(233)
plt.imshow(G1, cmap='gray')

plt.subplot(234)
plt.imshow(I, cmap='gray')
plt.subplot(235)
plt.imshow(k2, cmap='gray')
plt.subplot(236)
plt.imshow(G2, cmap='gray')

plt.show()

Si finalmente los valores de los filtros $k$ son reales, una red neuronal podria aprenderlos como los pesos $W$ de una capa Dense. El objetivo de la convolucion dentro de una red neuronal consiste en FRACCIONAR la imagen en sus partes constituyentes y aprender QUE elementos hacen que un objeto sea de una u otra categoria.


<img src="https://adeshpande3.github.io/assets/Cover.png">
<center> Tomada de: https://adeshpande3.github.io/assets/Cover.png</center>

## Max Pooling

Esta operacion tiene como objetivo reducir el tamanio de la senial a traves de la red neuronal de tal forma que las ultimas capas reciban una version de menor tamanio de la senial de las capas anteriores. De esta forma, se reduce el costo computacional del proceso de entrenamiento.

In [ ]:
def maxpooling(I, kernel_size):
    M, N = I.shape
    K = kernel_size #tamanio del pooling

    MK = M // K
    NL = N // K

    G = I[:MK*K, :NL*K].reshape(MK, K, NL, K).max(axis=(1, 3)) #maxpooling
    
    return G

In [ ]:
max_G1 = maxpooling(G1, 2)

plt.subplot(121)
plt.imshow(G1, cmap='gray')

plt.subplot(122)
plt.imshow(max_G1, cmap='gray')
plt.show()

<hr>

## Implementemos una red convolutiva en Keras

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Convolution2D(32, (3,3), activation='relu', input_shape=(28, 28, 1))) #capa convolutiva
model.add(keras.layers.MaxPooling2D(2,2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(units=10, activation=tf.nn.softmax))

In [ ]:
model.summary()

In [ ]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
print(training_images.shape)
print(training_labels.shape)
print("Categorias del dataset: ",np.unique(training_labels))


training_images =  training_images / 255.0
test_images = test_images / 255.0

training_images = training_images.reshape(60000,28, 28,1)
test_images = test_images.reshape(10000,28, 28,1)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)
model.evaluate(test_images, test_labels)

<hr>

## Trabajemos

1. Intenta editar las convoluciones. Cambiar las 32 a 16 o 64. Que impacto tiene esto en el tiempo de entrenamiento y la precision del modelo?.
2. Agregar una segunda capa de convolucion y una de maxpooling. Que impacto tiene esto en el tiempo de entrenamiento y la precision del modelo?